In [ ]:
import open3d
import argparse
import sys
from tqdm import tqdm_notebook as tqdm
import os
import time
import colorsys
import h5py
import datetime
import numpy as np
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F

sys.path.append('my_PointNet12/')
from my_PointNet12.model.chamfer import chamfer_batch
from my_PointNet12.clf import parse_args
from my_PointNet12.model.pointnet2 import PointNet2ClsMsg
from my_PointNet12.model.pointnet import PointNetCls, feature_transform_reguliarzer
from my_PointNet12.data_utils.ModelNetDataLoader import ModelNetDataLoader, load_data, class_names

import my_log as log
from my_utils import mkdir, num, print_mat, print_l2, sub_plot, norm_01

args = parse_args(True)
args.pretrain = 'checkpoints/clf-pointnet-0.89730-0076.pth'
args.batch_size = 8
test_data, test_label = load_data('my_PointNet12/experiment/data/modelnet40_ply_hdf5_2048/', train = False)
testDataset = ModelNetDataLoader(test_data, test_label)
testDataLoader = torch.utils.data.DataLoader(testDataset, batch_size=args.batch_size, shuffle=False)

log.info('Building Model',args.model_name)
if args.model_name == 'pointnet':
    num_class = 40
    model = PointNetCls(num_class,args.feature_transform).cuda()  
else:
    model = PointNet2ClsMsg().cuda()

torch.backends.cudnn.benchmark = True
model = torch.nn.DataParallel(model).cuda()
log.debug('Using gpu:',args.gpu)

    
if args.pretrain is None:
    log.err('No pretrain model')
else:
    log.debug('Loading pretrain model...')
    checkpoint = torch.load(args.pretrain)
    model.load_state_dict(checkpoint)
    model.eval()
    print('Done')

In [ ]:
epoches = 50
buf,buf2 = [],[]
for pts, gt in testDataLoader:
    batch_size = gt.size(0)
    gt = gt[:, 0].long().cuda()
    pts.requires_grad = False
    pts = pts.transpose(2, 1).cuda() # make channal first
    
    output, _ = model(pts)
    org_pred = output.data.max(1)[1]
    target = torch.from_numpy(np.array([0]* args.batch_size)).cuda()
    noise = torch.zeros_like(pts, requires_grad=True).cuda()
    opt = optim.Adam([noise], lr=0.01, betas=(0.9, 0.999))
    
    for i in tqdm(range(epoches)):        
        opt.zero_grad()
        pts_adv = pts + noise
        softmax, _ = model(pts_adv)
        loss_cross = nn.CrossEntropyLoss()(softmax, target)
        loss_l2 = (noise ** 2).sum() / batch_size
        loss_chamfer = chamfer_batch(pts_adv.transpose(2, 1), pts.transpose(2, 1))
        loss = loss_cross*2 + loss_chamfer + loss_l2 * 10
        loss.backward()
        opt.step()
            
        output, _ = model(pts_adv)
        adv_chocie = output.data.max(1)[1]
        is_target = num((adv_chocie == target).float().mean())
        not_orginal = num((adv_chocie == org_pred).float().mean())
        buf.append([num(loss), num(loss_cross), num(loss_chamfer), num(loss_l2)])
        buf2.append([is_target, not_orginal])
        del output,adv_chocie,is_target,not_orginal
    break

fig = plt.figure(figsize=(12, 2), dpi=80, facecolor='w', edgecolor='k')
for i,title in enumerate(['loss sum', 'CrossEntropyLoss ', 'ChamferLoss', 'L2']):
    ax = plt.subplot(1,5,i+1);
    plt.plot(np.array(buf)[:,i]);
    ax.title.set_text(title);
ax = plt.subplot(1,5,5);
plt.plot(np.array(buf2));
ax.title.set_text('is_target & not_orginal');
plt.show()

In [ ]:
# print(num(noise_clip).min(),num(noise_clip).max())
# noise_clip = torch.clamp_max(noise, eps)
# pts_adv = pts + noise_clip
aa = F.softmax(model(pts)[0], dim = 1)
bb = F.softmax(model(pts_adv)[0], dim = 1)

org = np.array(class_names)[num(aa.data.max(1)[1]).tolist()]
org_p = num(aa.data.max(1)[0]).tolist()

adv = np.array(class_names)[num(bb.data.max(1)[1]).tolist()]
adv_p = num(bb.data.max(1)[0]).tolist()

import pandas as pd
pd.DataFrame([org,org_p, adv,adv_p], index=['org','org_p','adv','adv_p'])

In [ ]:
def disp_pt_cloud(x,color = None):
    cloud = open3d.geometry.PointCloud()
    cloud.points = open3d.utility.Vector3dVector(x)
    if color is not None:
        cloud.colors = open3d.Vector3dVector(color)
    
    vis = open3d.visualization.VisualizerWithKeyCallback()
    vis.create_window(args.model_name, height=800, width=1200, left=200, top=0)
    opt = vis.get_render_option().background_color = np.asarray([0, 0, 0])
    vis.add_geometry(cloud)
    vis.register_key_callback(32, lambda vis: exit())
    vis.run()
    vis.destroy_window()

idx = 6
points_np = num(pts.detach().transpose(2, 1)[idx])
points_adv_np = num(pts_adv.detach().transpose(2, 1)[idx])
noise_np = num(noise.detach().transpose(2, 1)[idx])
disp_pt_cloud(points_np)
disp_pt_cloud(points_adv_np)
disp_pt_cloud(noise_np)

# FGSM 

In [ ]:
def disp_pt_cloud(x,channal_first = True):
    cloud = open3d.geometry.PointCloud()
    cloud.points = open3d.utility.Vector3dVector(x)

    vis = open3d.visualization.VisualizerWithKeyCallback()
    vis.create_window(args.model_name, height=800, width=800, left=200, top=0)
    opt = vis.get_render_option().background_color = np.asarray([0, 0, 0])
    vis.add_geometry(cloud)
    vis.register_key_callback(32, lambda vis: exit())
    vis.run()
    vis.destroy_window()

for eps in np.linspace(0,0.1,num=20):
    buf = []
    for pts, gt in testDataLoader:
        gt = gt[:, 0].long().cuda()
        pts = pts.transpose(2, 1).cuda() # make channal first
        pts.requires_grad = True
        
        softmax, _ = model(pts)
        org_pred = softmax.data.max(1)[1]
        loss = F.nll_loss(softmax, org_pred)
        model.zero_grad()
        loss.backward()
        pts_grad = pts.grad.data
        pts_adv = pts + eps * pts_grad.sign()

        chamfer_loss = num(chamfer_batch(pts_adv.transpose(2, 1), pts.transpose(2, 1)))
        softmax, _ = model(pts_adv)
        adv_pred = softmax.data.max(1)[1]

        is_org = num((adv_pred == org_pred).double().mean())
        log.info(eps=eps, is_org=is_org, chamfer = chamfer_loss)
        del pts, softmax, org_pred, chamfer_loss, adv_pred
        torch.cuda.empty_cache()
        break
#disp_pt_cloud(num(pts_adv[0]).transpose((1,0)))